## SQL
**Жилин Василий**

### Цель исследования

**Узнать, насколько эффективны однодневные рекламные кампании.**

### Задачи
* **Посчитайте затраты на каждый рекламный источник.**
* **Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег, с настольных ПК (тип устройства desktop).**
* **Введём новую метрику эффективности рекламы - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику.**


**Считаю исходные данные**

In [1]:
# импортирую библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# подключаюсь к базе данных
db_config = {'user': '********', # имя пользователя
'pwd': '********', # пароль
'host': '********',
'port': 1111, # порт подключения
'db': '********'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=True)

**Выведу первые строки таблиц**

In [3]:
# visits
query = '''
            select *
            from visits
            limit 5
        '''
pd.io.sql.read_sql(query, con = engine)

2020-03-23 15:34:03,633 INFO sqlalchemy.engine.base.Engine select version()
2020-03-23 15:34:03,648 INFO sqlalchemy.engine.base.Engine {}
2020-03-23 15:34:03,738 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-03-23 15:34:03,739 INFO sqlalchemy.engine.base.Engine {}
2020-03-23 15:34:03,837 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-23 15:34:03,840 INFO sqlalchemy.engine.base.Engine {}
2020-03-23 15:34:03,887 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-23 15:34:03,890 INFO sqlalchemy.engine.base.Engine {}
2020-03-23 15:34:03,953 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-03-23 15:34:03,954 INFO sqlalchemy.engine.base.Engine {}
2020-03-23 15:34:04,040 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00


In [4]:
# orders 
query = '''
            select *
            from orders
            limit 5
        '''
pd.io.sql.read_sql(query, con = engine)

2020-03-23 15:34:04,546 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-23 15:34:04,552 INFO sqlalchemy.engine.base.Engine {'name': '\n            select *\n            from orders\n            limit 5\n        '}
2020-03-23 15:34:04,720 INFO sqlalchemy.engine.base.Engine 
            select *
            from orders
            limit 5
        
2020-03-23 15:34:04,724 INFO sqlalchemy.engine.base.Engine {}


,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450


In [5]:
# advertisment_costs 
query = '''
            select *
            from advertisment_costs
            limit 5
        '''
pd.io.sql.read_sql(query, con = engine)

2020-03-23 15:34:04,923 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-23 15:34:04,924 INFO sqlalchemy.engine.base.Engine {'name': '\n            select *\n            from advertisment_costs\n            limit 5\n        '}
2020-03-23 15:34:05,102 INFO sqlalchemy.engine.base.Engine 
            select *
            from advertisment_costs
            limit 5
        
2020-03-23 15:34:05,106 INFO sqlalchemy.engine.base.Engine {}


,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53
3,4,1,2017-06-04,$55.00
4,5,1,2017-06-05,$57.08


**Задача 1. Посчитайте затраты на каждый рекламный источник.**

In [6]:
query = '''
            select sourceid, sum(costs) sum_cost
            from advertisment_costs
            group by sourceid
        '''
pd.io.sql.read_sql(query, con = engine)

2020-03-23 15:34:05,303 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-23 15:34:05,305 INFO sqlalchemy.engine.base.Engine {'name': '\n            select sourceid, sum(costs) sum_cost\n            from advertisment_costs\n            group by sourceid\n        '}
2020-03-23 15:34:05,458 INFO sqlalchemy.engine.base.Engine 
            select sourceid, sum(costs) sum_cost
            from advertisment_costs
            group by sourceid
        
2020-03-23 15:34:05,462 INFO sqlalchemy.engine.base.Engine {}


,sourceid,sum_cost
0,10,"$5,822.49"
1,9,"$5,517.49"
2,1,"$20,833.27"
3,5,"$51,757.10"
4,4,"$61,073.60"
5,2,"$42,806.04"
6,3,"$141,321.63"


**Задача2. Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов пользователями, которые хотя бы раз перешли из рекламного источника, на который было потрачено больше всего денег, с настольных ПК (тип устройства desktop).**

In [12]:
query = '''
            with source_sum_cost as
                (select sourceid, sum(costs) sum_cost
                from advertisment_costs
                group by sourceid)
            select sum (revenue) sum_revenue, max (revenue) max_revenue, min (revenue) min_revenue
            from orders
            where uid in
                (select distinct uid
                from visits
                where (device = 'desktop') and sourceid in
                    (select sourceid
                    from source_sum_cost
                    where sum_cost = (select max(sum_cost) from source_sum_cost)
                    )
                )
        '''
pd.io.sql.read_sql(query, con = engine)

2020-03-23 16:08:41,837 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-23 16:08:41,839 INFO sqlalchemy.engine.base.Engine {'name': "\n            with source_sum_cost as\n                (select sourceid, sum(costs) sum_cost\n                from advertisment_costs\n               ... (363 characters truncated) ... source_sum_cost\n                    where sum_cost = (select max(sum_cost) from source_sum_cost)\n                    )\n                )\n        "}
2020-03-23 16:08:41,969 INFO sqlalchemy.engine.base.Engine 
            with source_sum_cost as
                (select sourceid, sum(costs) sum_cost
                from advertisment_costs
                group by sourceid)
            select sum (revenue) sum_revenue, max (revenue) max_revenue, min (revenue) min_revenue
            from orders
            where uid in
                (selec

,sum_revenue,max_revenue,min_revenue
0,"$106,058.86","$2,633.28",$0.00


**Задача 3. Введём новую метрику эффективности рекламны - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику.**

In [8]:
query = '''
            select advertisment_costs.sourceid sourceid, advertisment_costs.dt dt,
                case when visiters_tbl.visiters is null then null
                else advertisment_costs.costs / visiters_tbl.visiters end costs_per_visiter
            from advertisment_costs
            left join
                (select sourceid, date_trunc('day', startts) dt, count(distinct uid) visiters
                from visits
                group by sourceid, date_trunc('day', startts)
                ) visiters_tbl on (advertisment_costs.sourceid = visiters_tbl.sourceid) and
                    (advertisment_costs.dt = visiters_tbl.dt)
        '''
pd.io.sql.read_sql(query, con = engine)

2020-03-23 15:34:06,770 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-23 15:34:06,773 INFO sqlalchemy.engine.base.Engine {'name': "\n            select advertisment_costs.sourceid sourceid, advertisment_costs.dt dt,\n                case when visiters_tbl.visiters is null then nu ... (359 characters truncated) ...  ) visiters_tbl on (advertisment_costs.sourceid = visiters_tbl.sourceid) and\n                    (advertisment_costs.dt = visiters_tbl.dt)\n        "}
2020-03-23 15:34:06,978 INFO sqlalchemy.engine.base.Engine 
            select advertisment_costs.sourceid sourceid, advertisment_costs.dt dt,
                case when visiters_tbl.visiters is null then null
                else advertisment_costs.costs / visiters_tbl.visiters end costs_per_visiter
            from advertisment_costs
            left join
                (select sourceid, da

,sourceid,dt,costs_per_visiter
0,1,2017-06-01,$1.27
1,1,2017-06-02,$1.24
2,1,2017-06-03,$0.98
3,1,2017-06-04,$1.25
4,1,2017-06-05,$0.79
5,1,2017-06-06,$0.45
6,1,2017-06-07,$0.62
7,1,2017-06-08,$0.68
8,1,2017-06-09,$0.73
9,1,2017-06-10,$1.66


*метрика посчитана; для выводов об эффективности маркетинговых кампаний требуется дальнейший анализ: построение наглядных графиков, сопоставление таких расходов с выручкой от одного посетителя, маркетинговая воронка, сравнение показателей для различных рекламных источников...*